In [1]:
from new_model.model_utils import resnet_shorten, hybrid_LSTM_training, _classifier, convolutional_block
from keras.layers import LSTM, Reshape, Input, Conv2D, MaxPooling2D, Lambda
from keras.models import Model
import keras
from training_utils import save_model, DataGenerator, generate_dataset

import time
import pickle

Using TensorFlow backend.


# Global constant

In [2]:
IMG_SHAPE = (199, 265, 1)
LSTM_DIM_HIDDEN = 64
LEN_SPATIAL_HISTORY = 4
NUM_CLASS = 73

# Name of previously trained weights

In [3]:
shared_encoder_file = "shared_encoder_resnet8_2019_05_22_11_04.h5"
# separate_encoder_files = ["sep_encoder_%d_2019_05_22_11_04.h5" % i for i in range(LEN_SPATIAL_HISTORY)]
# shared_lstm_file = ".p"
shared_classifier_file = "classifier_2019_05_22_11_04.h5"

___

# 2.Model Definition

**------------------------**
## 2.1 Encoder

### 2.1.1 Shared encoder

In [4]:
shared_encoder = resnet_shorten(IMG_SHAPE, model_name="shared_encoder")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
shared_encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 199, 265, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 133, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 66, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNormalization)     (None, 49, 66, 32)   128         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
activation

### 2.1.2 Separate encoder

This is a convolution block of ResNet.

In [6]:
def _separate_encoder(input_shape, model_name):
    """
    Create a model from the function named "convolutional_block". This model is later used as a layer
    in the full hybrid model
    
    Input: 
        input_shape (tuple): shape of feature vectors created by shared_encoder
        num_filters (list): number of filters of each Conv2D layer of this model
        strides (list): size of strides of each Conv2D layer
        stage: must set to be None
        model_name (str):
        
    Output:
        keras Model instance
    """
    
    # define input
    X_input = Input(shape=input_shape)
    
    X = Conv2D(128, (3, 3), padding='valid', strides=(2, 2))(X_input)
    X = MaxPooling2D()(X)
    X = Conv2D(256, (3, 3), padding='valid', strides=(2, 2))(X)
    X = Reshape((1, -1))(X)
    # define model
    model = Model(inputs=[X_input], outputs=[X], name=model_name)
    
    return model
    

In [7]:
# define config of separate encoder
s_input_shape = (13, 17, 64)  # shape of output of shared encoder

sep_encoder_list = [_separate_encoder(s_input_shape, model_name="sep_en_%d" % i) 
                    for i in range(LEN_SPATIAL_HISTORY)]

In [8]:
sep_encoder_list[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 13, 17, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 4, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 256)         295168    
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 256)            0         
Total params: 369,024
Trainable params: 369,024
Non-trainable params: 0
_________________________________________________________________


**------------------------**
## 2.2 Decoder

### 2.2.1 LSTM cell

In [9]:
LSTM_cell = LSTM(LSTM_DIM_HIDDEN, return_sequences=True)
LSTM_cell_2 = LSTM(LSTM_DIM_HIDDEN, return_sequences=True)

### 2.2.1 Define Classifier

In [10]:
classifier = _classifier(input_shape=(LSTM_DIM_HIDDEN, ), num_class=NUM_CLASS)

In [11]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_2 (Dense)              (None, 73)                18761     
_________________________________________________________________
activation_5 (Activation)    (None, 73)                0         
Total params: 35,401
Trainable params: 35,401
Non-trainable params: 0
_________________________________________________________________


## 2.3 Full Model

In [12]:
def model_v3(image_shape, shared_encoder, sep_encoder_list, LSTM_cell_list, classifier, Ty):
    """
    Architect: image -> shared_encoder -> separate_encoder -> lstm_1 -> lstm_2 -> shared_classifier -> y
    
    Input:
        image_shape (tuple): shape of input image
        shared_encoder (keras.Model): shared model used to extract low level feature vector from input image
        sep_encoder_list (list): list of keras.Model storing separate encoder
        LSTM_cell (keras.layers)
        classifier (keras.Model): shared classifier to predict class of steering angle
        Ty (int): length of spatial history
        
    """
    # Input layer
    X_input_list = [Input(shape=image_shape) for i in range(Ty)]
    
    # pass each input through shared encoder
    shared_encoded_X = [shared_encoder(X) for X in X_input_list]
    
    # pass each encoded_X through its own convolution block
    separate_encoded_X = [separate_encoder(X) 
                          for separate_encoder, X in zip(sep_encoder_list, shared_encoded_X)]
    
    # concatenate encoded vector
    X = keras.layers.concatenate(separate_encoded_X, axis=1)
    
    # 1st LSTM layer
    X = LSTM_cell_list[0](X)  # output shape (LEN_SPATIAL_HISTORY, LSTM_DIM_HIDDEN)
    X = LSTM_cell_list[1](X)
    
    # propagate through classifier
    outputs = []
    for i in range(LEN_SPATIAL_HISTORY):
        X_i = Lambda(lambda x: x[:,i,:])(X)  # slice tensor X
        # invoke classifier
        outputs.append(classifier(X_i))
    
    # define model
    model = Model(inputs=X_input_list, outputs=outputs)
    return model



In [13]:
hybrid_model = model_v3(IMG_SHAPE, shared_encoder, sep_encoder_list, [LSTM_cell, LSTM_cell_2], classifier, LEN_SPATIAL_HISTORY)

In [14]:
hybrid_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 199, 265, 1)  0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 199, 265, 1)  0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 199, 265, 1)  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 199, 265, 1)  0                                            
__________________________________________________________________________________________________
shared_enc

## 2.3 Load weights & compile model

In [16]:
# Load shared_encoder
shared_encoder.load_weights("./new_model/weights/shared_encoder/%s" % shared_encoder_file, by_name=True)

# # Load shared LSTM
# with open('./new_model/weights/shared_lstm/%s' % shared_lstm_file, 'rb') as fp:
#     lstm_weights_dict = pickle.load(fp)
    
# lstm_weights = []
# for k in lstm_weights_dict.keys():
#     lstm_weights.append(lstm_weights_dict[k])
    
# LSTM_cell.set_weights(lstm_weights)

# # Load separate encoder
# for sep_enc_file, sep_enc in zip(separate_encoder_files, sep_encoder_list):
#     sep_enc.load_weights("./new_model/weights/separate_encoder/%s" % sep_enc_file)

# Load shared classifier
# classifier.load_weights("./new_model/weights/shared_classifier/%s" % shared_classifier_file)

In [30]:
otim = keras.optimizers.Adam(lr=0.35, decay=0.001)
hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

---

# 3. Training

In [18]:
batch_size = 32

param_train = {'img_shape': IMG_SHAPE, 
             'Ty': LEN_SPATIAL_HISTORY, 
             'num_class': NUM_CLASS, 
             'batch_size': batch_size, 
             'shuffle': True, 
             'additional_input_for_LSTM': False, 
             'LSTM_dim_hidden_states': LSTM_DIM_HIDDEN}

train_gen = DataGenerator("./new_data/widthen_bin_training_CH2_only.csv", **param_train)

param_val = {'img_shape': IMG_SHAPE, 
             'num_class': NUM_CLASS, 
             'Ty': LEN_SPATIAL_HISTORY, 
             'LSTM_dim_hidden_states': LSTM_DIM_HIDDEN, 
             'additional_input_for_LSTM': False,
             'color_img': False}
X_val, y_val = generate_dataset("./new_data/widthen_bin_validation_CH2_only.csv", **param_val)

In [31]:
time_str = time.strftime("%Y_%m_%d_%H_%M")
tb_callback = keras.callbacks.TensorBoard(log_dir='./logs/' + time_str,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')


In [32]:
hybrid_model.fit_generator(train_gen,
                           epochs=30,
                           validation_data=(X_val, y_val),
                           initial_epoch=20,
                           callbacks=[tb_callback])

Epoch 21/30
569/569 [==============================] - 234s 411ms/step - loss: 8.7410 - model_1_loss: 2.1067 - model_1_acc: 0.3226 - model_1_acc_1: 0.3299 - model_1_acc_2: 0.3479 - model_1_acc_3: 0.3791 - val_loss: 8.8232 - val_model_1_loss: 2.1259 - val_model_1_acc: 0.3235 - val_model_1_acc_1: 0.3368 - val_model_1_acc_2: 0.3486 - val_model_1_acc_3: 0.3738
Epoch 22/30
569/569 [==============================] - 232s 407ms/step - loss: 8.7225 - model_1_loss: 2.0974 - model_1_acc: 0.3238 - model_1_acc_1: 0.3343 - model_1_acc_2: 0.3521 - model_1_acc_3: 0.3836 - val_loss: 8.8362 - val_model_1_loss: 2.1403 - val_model_1_acc: 0.2948 - val_model_1_acc_1: 0.3146 - val_model_1_acc_2: 0.3452 - val_model_1_acc_3: 0.3501
Epoch 23/30
569/569 [==============================] - 234s 411ms/step - loss: 8.7015 - model_1_loss: 2.0854 - model_1_acc: 0.3255 - model_1_acc_1: 0.3349 - model_1_acc_2: 0.3518 - model_1_acc_3: 0.3864 - val_loss: 8.7709 - val_model_1_loss: 2.1067 - val_model_1_acc: 0.3314 - val_m

# 4. Save weights

In [33]:
# save shared_encoder 
shared_encoder.save_weights("./new_model/weights/shared_encoder/shared_encoder_resnet8_%s.h5" % time_str)

# save shared_lstm
save_lstm(LSTM_cell, 1)
save_lstm(LSTM_cell_2, 2)

# save separate encoder
for i, sep_encoder in enumerate(sep_encoder_list):
    sep_encoder.save_weights("./new_model/weights/separate_encoder/sep_encoder_v3_%d_%s.h5" % (i, time_str))

# save classifier
classifier.save_weights("./new_model/weights/shared_classifier/classifier_%s.h5" % time_str)